In [1]:
import os
import gzip
from Bio import SeqIO
from collections import defaultdict as dd
from functools import wraps
from time import perf_counter_ns

# Genome loading exploration

## Timing helper

In [16]:
def timeit(f):
    @wraps(f)
    def wrapper(*args, **kwargs):
        start = perf_counter_ns()
        result = f(*args, **kwargs)
        print(f"{f}({args[0]}) took {(perf_counter_ns() - start) / 10 ** 6:.2f}ms")
        return result
    return wrapper

## Basic file search

In [3]:
ISEULT_GENOMES_PATH = "/no_backup/rg/ileahy/protists"

In [4]:
subdirs = os.listdir(ISEULT_GENOMES_PATH)
len(subdirs)

103

In [5]:
os.listdir(ISEULT_GENOMES_PATH  + '/' + subdirs[0])

['Nitzschia_sp._DOCU1_genomic.fa',
 'Nitzschia_sp._DOCU1_assembly_stats.txt',
 'Nitzschia_sp._DOCU1_genomic.lengths',
 'Nitzschia_sp._DOCU1_genomic.fna.nhr',
 'Nitzschia_sp._DOCU1_genomic.index',
 'Nitzschia_sp._DOCU1_genomic.gbff.gz',
 'Nitzschia_sp._DOCU1_genomic.fna.nsq',
 'output',
 'Nitzschia_sp._DOCU1_genomic.gff.gz',
 'Nitzschia_sp._DOCU1_genomic.fna.gz',
 'Nitzschia_sp._DOCU1_genomic.fa.ssi',
 'Nitzschia_sp._DOCU1_genomic.fna.nsd',
 'Nitzschia_sp._DOCU1_genomic.fna.nin',
 'Nitzschia_sp._DOCU1_genomic.fna.nsi']

In [6]:
fastas = {i: f"{ISEULT_GENOMES_PATH}/{i}/{i}_genomic.fna.gz" for i in subdirs}
len(fastas)

103

In [7]:
first = list(fastas.items())[0]
print(first)

('Nitzschia_sp._DOCU1', '/no_backup/rg/ileahy/protists/Nitzschia_sp._DOCU1/Nitzschia_sp._DOCU1_genomic.fna.gz')


## Manual file opening

In [8]:
with gzip.open(first[1], mode = 'rt') as f:
    lines = [line.strip() for line in f.readlines()]

print(lines[:5])

['>OX595985.1 Nitzschia sp. DOCU1 genome assembly, chromosome: 1', 'accctaaccctaaccctaaccctaaccctaaccctaaccctaaccctaaccctaaccctaaccctaaccctaaccctaac', 'cctaaccctaaccctaaccctaaccctaaccctaaccctaaccctaaccctaaccctaaccctaaccctaaccctaaccc', 'taaccctaaccctaaccctaaccctaaccctaaccctaaccctaaccctaaccctaaccctaaccctaaccctaacccta', 'accctaaccctaaccctaaccctaaccctaaccctaaccctaaccctaaccctaaccctaaccctaaccctaaccctaac']


In [9]:
with gzip.open(first[1], 'rt') as f:
    seqs_first = SeqIO.parse(f, "fasta")
    print(type(seqs_first))
    chromosomes_first = [seq for seq in seqs_first if "chromosome" in seq.description]

<class 'Bio.SeqIO.FastaIO.FastaIterator'>


## Timing iterator vs. list reading

In [24]:
@timeit
def read_genome_iter(path):
    with gzip.open(path, "rt") as f:
        return SeqIO.parse(f, "fasta")

In [26]:
@timeit
def read_genome_list(path):
    with gzip.open(path, "rt") as f:
        return list(SeqIO.parse(f, "fasta"))

In [32]:
genome_it = read_genome_iter(list(fastas.items())[1][1])
# takes about 1ms

<function read_genome_iter at 0x7fa5c87c54e0>(/no_backup/rg/ileahy/protists/Neospora_caninum_Liverpool/Neospora_caninum_Liverpool_genomic.fna.gz) took 0.59ms


In [37]:
genome_list = read_genome_list(list(fastas.items())[1][1])
# takes about 400ms

<function read_genome_list at 0x7fa5c87c5bc0>(/no_backup/rg/ileahy/protists/Neospora_caninum_Liverpool/Neospora_caninum_Liverpool_genomic.fna.gz) took 389.57ms


In [40]:
# next(genome_it)
# the iterator only works while the file is open

We should exploit FASTA iterators for large-scale analysis, they should work much faster than converting to a list

In [9]:
len(chromosomes_first)

17

## Some kmer exploration (not too relevant for now)

In [10]:
def kmers(seq, k):
    counts = dd(int)
    total_count = 0
    for i in range(0, len(seq) - k, k):
        kmer = seq.upper()[i:i+k]
        counts[kmer] += 1
        total_count += 1
    freqs = {kmer: count / total_count for kmer, count in counts.items()}
    return freqs

In [18]:
kmers("CGATCAGCTGATCAGCTACGATTCTCAGCAT", 3)

{'CGA': 0.2,
 'TCA': 0.2,
 'GCT': 0.1,
 'GAT': 0.1,
 'CAG': 0.1,
 'CTA': 0.1,
 'TTC': 0.1,
 'GCA': 0.1}

In [ ]:
# kmers(chromosomes_first[0].seq, 3)